1_Instalación de Librerías (Si es necesario en Colab)

In [18]:
# N: Esta celda es útil si estás ejecutando esto en un nuevo entorno de Colab
# N: Si ya tienes las librerías instaladas o tu runtime las incluye, puedes saltarla.
!pip install xgboost scikit-learn pandas numpy joblib

2_Importación de Librerías

In [19]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
import joblib # Para guardar y cargar modelos

# N: Si tienes warnings de librerías que no afectan la ejecución, puedes ignorarlos
import warnings
warnings.filterwarnings('ignore')

3_Carga de Datos

In [20]:
# N: Asegúrate de que estos archivos CSV estén disponibles en tu entorno de Colab.
# N: Puedes subirlos manualmente a la sección 'Files' de Colab o montando Google Drive.

# Cargar datasets
df_ush = pd.read_csv('USH_completo_renombrado.csv')
df_rg = pd.read_csv('RG_completo_renombrado.csv')

# Mostrar las primeras filas y la información para verificar
print("Datos de Ushuaia (df_ush.head()):")
print(df_ush.head())
print("\nInformación de Ushuaia (df_ush.info()):")
df_ush.info()

print("\nDatos de Río Grande (df_rg.head()):")
print(df_rg.head())
print("\nInformación de Río Grande (df_rg.info()):")
df_rg.info()

# N: Verificar que las columnas de mes_X estén presentes y sean numéricas (0 o 1)
# N: y que no haya NaNs en las columnas que se usarán para entrenar.
# N: Por ejemplo:
print("\nValores nulos en df_ush después de la carga:")
print(df_ush.isnull().sum()[df_ush.isnull().sum() > 0])

print("\nValores nulos en df_rg después de la carga:")
print(df_rg.isnull().sum()[df_rg.isnull().sum() > 0])

Datos de Ushuaia (df_ush.head()):
   anio  temp_max  temp_min  temp_media  lluvia_mm  dias_nieve   toh   top  \
0  2009      10.8       9.4        10.1       30.8           0  66.1  56.0   
1  2009       9.5       8.3         8.9       29.0           0  54.5  45.0   
2  2009       9.1       7.9         8.5       64.6           1  39.2  30.7   
3  2009       6.1       5.2         5.7       86.2           1  31.5  24.3   
4  2009       4.7       2.0         2.6       59.2           4  22.1  16.5   

   ent_san_sebastian  aero_ush  ...  mes_diciembre  mes_enero  mes_febrero  \
0              26309   33589.0  ...          False       True        False   
1              15123   31671.0  ...          False      False         True   
2              14684   20569.0  ...          False      False        False   
3              13319   15312.0  ...          False      False        False   
4              12729    9878.0  ...          False      False        False   

   mes_julio  mes_junio  mes

4_ Definición de Características (Features) y Variables Objetivo (Targets)

In [21]:
# N: Asegúrate de que la lista de características aquí coincida EXACTAMENTE
# N: con las columnas resultantes de tu preprocesamiento y EDA.

# Características comunes para ambos datasets
common_features = [
    'anio',
    'temp_max', 'temp_min', 'temp_media', 'lluvia_mm', 'dias_nieve',
    'ent_san_sebastian', 'aero_ush', 'aero_rg',
    'mes_enero', 'mes_febrero', 'mes_marzo', 'mes_abril', 'mes_mayo', 'mes_junio',
    'mes_julio', 'mes_agosto', 'mes_septiembre', 'mes_octubre', 'mes_noviembre', 'mes_diciembre'
]

# Variables objetivo
target_toh = 'toh'
target_top = 'top'

# --- Ajustes específicos por dataset ---

# Para Ushuaia: Usamos todas las características comunes
features_ush = common_features

# Para Río Grande: Quitamos 'mes_julio' como se decidió en el tuning final
features_rg = [f for f in common_features if f != 'mes_julio']

# N: Verificación final de que las columnas existen en los DataFrames
print("\n--- Verificación de Características en DataFrames ---")
missing_ush_features = [f for f in features_ush if f not in df_ush.columns]
if missing_ush_features:
    print(f"ATENCIÓN: Faltan las siguientes características en df_ush: {missing_ush_features}")
else:
    print("Todas las características de Ushuaia están presentes.")

missing_rg_features = [f for f in features_rg if f not in df_rg.columns]
if missing_rg_features:
    print(f"ATENCIÓN: Faltan las siguientes características en df_rg: {missing_rg_features}")
else:
    print("Todas las características de Río Grande están presentes.")

if target_toh not in df_ush.columns or target_top not in df_ush.columns:
    print("ATENCIÓN: Las columnas de target (toh, top) no están en df_ush.")
else:
    print("Variables objetivo (toh, top) presentes en df_ush.")

if target_toh not in df_rg.columns or target_top not in df_rg.columns:
    print("ATENCIÓN: Las columnas de target (toh, top) no están en df_rg.")
else:
    print("Variables objetivo (toh, top) presentes en df_rg.")


--- Verificación de Características en DataFrames ---
Todas las características de Ushuaia están presentes.
Todas las características de Río Grande están presentes.
Variables objetivo (toh, top) presentes en df_ush.
Variables objetivo (toh, top) presentes en df_rg.


 5_ Definición de Parámetros que exploró GridSearchCV



In [22]:
# N: Aquí se definen los mejores hiperparámetros encontrados en las ejecuciones previas de GridSearchCV.
# N: Esto nos permite entrenar los modelos directamente con la configuración óptima,
# N: sin la necesidad de volver a ejecutar la búsqueda exhaustiva (que es muy lenta).

# Mejores hiperparámetros para Ushuaia - TOH%
best_params_ush_toh = {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 300, 'subsample': 0.7}

# Mejores hiperparámetros para Ushuaia - TOP%
best_params_ush_top = {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 200, 'subsample': 0.7}

# Mejores hiperparámetros para Río Grande - TOH% (sin mes_julio)
best_params_rg_toh_no_julio = {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 200, 'subsample': 0.8}

# Mejores hiperparámetros para Río Grande - TOP% (sin mes_julio)
best_params_rg_top_no_julio = {'colsample_bytree': 0.9, 'gamma': 0, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.9}

print("Hiperparámetros óptimos definidos para cada modelo.")

Hiperparámetros óptimos definidos para cada modelo.


6_ Función de Entrenamiento y Evaluación del Modelo

In [23]:
# N: Esta función ahora acepta 'best_params' y entrena el modelo directamente,
# N: sin ejecutar GridSearchCV, lo que acelera mucho el proceso.

def train_and_evaluate_model(df, features, target_col, city_name, model_type, best_params):
    """
    Entrena y evalúa un modelo XGBoost con los hiperparámetros dados.

    Args:
        df (pd.DataFrame): DataFrame con los datos completos (entrenamiento + prueba).
        features (list): Lista de nombres de columnas a usar como características.
        target_col (str): Nombre de la columna objetivo.
        city_name (str): Nombre de la ciudad (para logs).
        model_type (str): Tipo de modelo (TOH%/TOP%) (para logs).
        best_params (dict): Diccionario de hiperparámetros óptimos para el modelo.

    Returns:
        tuple: (best_model (xgb.XGBRegressor), metrics (dict))
    """
    print(f"\n--- Entrenando modelo para {city_name} - {model_type} con parámetros óptimos ---")

    # Asegurarse de que las columnas de mes_X sean de tipo int (0/1)
    for col in df.columns:
        if col.startswith('mes_') and df[col].dtype == 'bool':
            df[col] = df[col].astype(int)

    # N: Asegúrate de que las columnas de features existen antes de seleccionarlas
    # N: y de que no haya NaNs en las features o el target.
    df_filtered = df.dropna(subset=features + [target_col]).copy()
    if df_filtered.empty:
        print(f"Error: No hay datos suficientes para {city_name} - {model_type} después de eliminar NaNs.")
        return None, None

    X = df_filtered[features]
    y = df_filtered[target_col]

    # Dividir datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print(f"Tamaño de X_train: {X_train.shape}, X_test: {X_test.shape}")

    # N: Instancia el modelo XGBoost directamente con los mejores hiperparámetros
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1, **best_params)

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Evaluar el modelo en el conjunto de prueba
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    r2 = r2_score(y_test, y_pred)

    print(f"Resultados de evaluación para {city_name} {model_type} (conjunto de prueba):")
    print(f"  RMSE: {rmse:.2f}")
    print(f"  MAPE: {mape:.2f}%")
    print(f"  R2 Score: {r2:.2f}")

    return model, {'rmse': rmse, 'mape': mape, 'r2': r2}

In [24]:
# N: Asegúrate de que la carpeta donde guardarás los modelos exista o se cree automáticamente.
# N: En Colab, los archivos se guardan en el directorio raíz por defecto.

# Diccionario para almacenar los mejores modelos y sus métricas
trained_models = {}
evaluation_results = {}

# --- Modelo: Ushuaia TOH% ---
best_model_ush_toh, metrics_ush_toh = train_and_evaluate_model(df_ush, features_ush, target_toh, "Ushuaia", "TOH%", best_params_ush_toh)
if best_model_ush_toh:
    trained_models['ush_toh'] = best_model_ush_toh
    evaluation_results['ush_toh'] = metrics_ush_toh
    joblib.dump(best_model_ush_toh, 'best_model_ush_toh.joblib')
    print("Modelo best_model_ush_toh.joblib guardado.")

# --- Modelo: Ushuaia TOP% ---
best_model_ush_top, metrics_ush_top = train_and_evaluate_model(df_ush, features_ush, target_top, "Ushuaia", "TOP%", best_params_ush_top)
if best_model_ush_top:
    trained_models['ush_top'] = best_model_ush_top
    evaluation_results['ush_top'] = metrics_ush_top
    joblib.dump(best_model_ush_top, 'best_model_ush_top.joblib')
    print("Modelo best_model_ush_top.joblib guardado.")

# --- Modelo: Río Grande TOH% (sin mes_julio) ---
# N: Aquí se aplica la exclusión de mes_julio para RG a través de 'features_rg'
best_model_rg_toh_no_julio, metrics_rg_toh_no_julio = train_and_evaluate_model(df_rg, features_rg, target_toh, "Río Grande", "TOH% (sin julio)", best_params_rg_toh_no_julio)
if best_model_rg_toh_no_julio:
    trained_models['rg_toh_no_julio'] = best_model_rg_toh_no_julio
    evaluation_results['rg_toh_no_julio'] = metrics_rg_toh_no_julio
    joblib.dump(best_model_rg_toh_no_julio, 'best_model_rg_toh_no_julio.joblib')
    print("Modelo best_model_rg_toh_no_julio.joblib guardado.")

# --- Modelo: Río Grande TOP% (sin mes_julio) ---
# N: Aquí se aplica la exclusión de mes_julio para RG a través de 'features_rg'
best_model_rg_top_no_julio, metrics_rg_top_no_julio = train_and_evaluate_model(df_rg, features_rg, target_top, "Río Grande", "TOP% (sin julio)", best_params_rg_top_no_julio)
if best_model_rg_top_no_julio:
    trained_models['rg_top_no_julio'] = best_model_rg_top_no_julio
    evaluation_results['rg_top_no_julio'] = metrics_rg_top_no_julio
    joblib.dump(best_model_rg_top_no_julio, 'best_model_rg_top_no_julio.joblib')
    print("Modelo best_model_rg_top_no_julio.joblib guardado.")

print("\n--- Resumen de Métricas de Evaluación Finales ---")
for model_name, metrics in evaluation_results.items():
    print(f"  {model_name}: RMSE={metrics['rmse']:.2f}, MAPE={metrics['mape']:.2f}%, R2={metrics['r2']:.2f}")


--- Entrenando modelo para Ushuaia - TOH% con parámetros óptimos ---
Tamaño de X_train: (132, 21), X_test: (34, 21)
Resultados de evaluación para Ushuaia TOH% (conjunto de prueba):
  RMSE: 6.00
  MAPE: 12.22%
  R2 Score: 0.92
Modelo best_model_ush_toh.joblib guardado.

--- Entrenando modelo para Ushuaia - TOP% con parámetros óptimos ---
Tamaño de X_train: (132, 21), X_test: (34, 21)
Resultados de evaluación para Ushuaia TOP% (conjunto de prueba):
  RMSE: 4.54
  MAPE: 15.40%
  R2 Score: 0.94
Modelo best_model_ush_top.joblib guardado.

--- Entrenando modelo para Río Grande - TOH% (sin julio) con parámetros óptimos ---
Tamaño de X_train: (132, 20), X_test: (34, 20)
Resultados de evaluación para Río Grande TOH% (sin julio) (conjunto de prueba):
  RMSE: 4.13
  MAPE: 9.30%
  R2 Score: 0.92
Modelo best_model_rg_toh_no_julio.joblib guardado.

--- Entrenando modelo para Río Grande - TOP% (sin julio) con parámetros óptimos ---
Tamaño de X_train: (132, 20), X_test: (34, 20)
Resultados de evaluac

7_ Función de predicción más amigable

In [25]:
# N: Esta función será muy útil para usar los modelos una vez entrenados.
# N: Permite pasarle nuevos datos y obtener la predicción,
# N: manejando automáticamente la columna 'mes_julio' para RG si se presenta.

def make_prediction(model_key, new_data_df):
    """
    Realiza una predicción utilizando uno de los modelos entrenados.

    Args:
        model_key (str): Clave del modelo a usar ('ush_toh', 'ush_top', 'rg_toh_no_julio', 'rg_top_no_julio').
        new_data_df (pd.DataFrame): DataFrame con los datos para predecir (una o más filas).
                                     Debe contener todas las columnas de características
                                     que podrían ser relevantes para el modelo.
                                     Si un mes_X es TRUE, asegúrate de que el resto de mes_Y sean FALSE.

    Returns:
        np.array: Array con las predicciones.
    """
    if model_key not in trained_models:
        print(f"Error: Modelo '{model_key}' no encontrado. Asegúrate de que los modelos estén entrenados y en el diccionario 'trained_models'.")
        return None

    model = trained_models[model_key]

    # Determinar las características esperadas por el modelo específico
    # N: Usamos la lista global 'features_ush' o 'features_rg'
    if 'rg' in model_key:
        expected_features = features_rg # Esta lista ya NO incluye 'mes_julio'
    else:
        expected_features = features_ush # Esta lista SÍ incluye 'mes_julio'

    # Preparar el DataFrame de entrada para la predicción
    # Se crea un DataFrame con las columnas esperadas y se inicializan a 0.
    # Luego se copian los valores de 'new_data_df' a este DataFrame.
    processed_data = pd.DataFrame(0, index=new_data_df.index, columns=expected_features)

    for col in expected_features:
        if col in new_data_df.columns:
            processed_data[col] = new_data_df[col]
        # N: Si una columna esperada no está en new_data_df, se mantiene como 0, lo cual es correcto para features One-Hot Encoded.
        # N: Para otras features numéricas (temp, lluvia), esto podría requerir imputación si new_data_df tiene NaNs.

    # Asegurarse de que el orden de las columnas sea el mismo que el usado en el entrenamiento
    processed_data = processed_data[expected_features]

    # Realizar la predicción
    predictions = model.predict(processed_data)

    return predictions

8_ Ejemplos de predicción

In [26]:
print("\n--- Ejemplos de Predicción con Datos Específicos ---")

# Datos para Ushuaia (Diciembre 2022)
# N: Asegúrate que las columnas booleanas (meses) estén correctamente interpretadas como True/False
# N: Aquí 'mes_diciembre' es True, el resto False
ush_test_data = pd.DataFrame({
    'anio': [2022],
    'temp_max': [14.5],
    'temp_min': [5.2],
    'temp_media': [9.5],
    'lluvia_mm': [58.2],
    'dias_nieve': [0],
    'toh': [59.7], # N: Este valor se ignorará para la predicción
    'top': [45.3], # N: Este valor se ignorará para la predicción
    'ent_san_sebastian': [17719],
    'aero_ush': [54968.0],
    'aero_rg': [5630],
    'mes_abril': [False], 'mes_agosto': [False], 'mes_diciembre': [True],
    'mes_enero': [False], 'mes_febrero': [False], 'mes_julio': [False],
    'mes_junio': [False], 'mes_marzo': [False], 'mes_mayo': [False],
    'mes_noviembre': [False], 'mes_octubre': [False], 'mes_septiembre': [False]
})

pred_ush_toh = make_prediction('ush_toh', ush_test_data)
print(f"Predicción TOH% Ushuaia (Diciembre 2022): {pred_ush_toh[0]:.2f}%")

pred_ush_top = make_prediction('ush_top', ush_test_data)
print(f"Predicción TOP% Ushuaia (Diciembre 2022): {pred_ush_top[0]:.2f}%")


# Datos para Río Grande (Diciembre 2022)
# N: Asegúrate que las columnas booleanas (meses) estén correctamente interpretadas como True/False
# N: Aquí 'mes_diciembre' es True, el resto False
rg_test_data = pd.DataFrame({
    'anio': [2022],
    'temp_max': [15.9],
    'temp_min': [4.5],
    'temp_media': [10.3],
    'lluvia_mm': [17.5],
    'dias_nieve': [0],
    'toh': [28.2], # N: Este valor se ignorará para la predicción
    'top': [16.0], # N: Este valor se ignorará para la predicción
    'ent_san_sebastian': [17719.0],
    'aero_ush': [54968.0],
    'aero_rg': [5630],
    'mes_abril': [False], 'mes_agosto': [False], 'mes_diciembre': [True],
    'mes_enero': [False], 'mes_febrero': [False], 'mes_julio': [False], # N: 'mes_julio' estará presente pero será ignorado por el modelo RG
    'mes_junio': [False], 'mes_marzo': [False], 'mes_mayo': [False],
    'mes_noviembre': [False], 'mes_octubre': [False], 'mes_septiembre': [False]
})

pred_rg_toh = make_prediction('rg_toh_no_julio', rg_test_data)
print(f"Predicción TOH% Río Grande (Diciembre 2022, 'mes_julio' presente en input, pero ignorado por el modelo RG): {pred_rg_toh[0]:.2f}%")

pred_rg_top = make_prediction('rg_top_no_julio', rg_test_data)
print(f"Predicción TOP% Río Grande (Diciembre 2022, 'mes_julio' presente en input, pero ignorado por el modelo RG): {pred_rg_top[0]:.2f}%")


--- Ejemplos de Predicción con Datos Específicos ---
Predicción TOH% Ushuaia (Diciembre 2022): 60.93%
Predicción TOP% Ushuaia (Diciembre 2022): 45.54%
Predicción TOH% Río Grande (Diciembre 2022, 'mes_julio' presente en input, pero ignorado por el modelo RG): 28.00%
Predicción TOP% Río Grande (Diciembre 2022, 'mes_julio' presente en input, pero ignorado por el modelo RG): 17.28%


###Para Ushuaia (Diciembre 2022):

Valores Reales (de tus datos de entrada):

TOH%: 59.7%  
TOP%: 45.3%  
  
Tus Predicciones:

Predicción TOH% Ushuaia: 60.93%  
Predicción TOP% Ushuaia: 45.54%  
   
###Para Río Grande (Diciembre 2022):

Valores Reales (de tus datos de entrada):

TOH%: 28.2%  
TOP%: 16.0%  
  
Tus Predicciones:
Predicción TOH% Río Grande: 28.00%  
Predicción TOP% Río Grande: 17.28%  